<a href="https://colab.research.google.com/github/zeerafle/sitting-posture/blob/master/notebooks/image_generation/sdxl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

!apt -y update -qq
!apt -y install -qq aria2
# !pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
# !pip install -q xformers==0.0.16 triton==2.0.0 -U
# !pip install -q mediapipe==0.9.1.0 addict yapf fvcore omegaconf

!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /content/ComfyUI
!pip install -q -r requirements.txt
!git reset --hard

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)


In [ ]:
from google.colab import userdata
import os

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['CIVITAI_TOKEN'] = userdata.get('CIVITAI_TOKEN')

In [ ]:
%%bash

# Set up environment
COMFYUI_DIR=/content/ComfyUI

NODES=(
    "https://github.com/ltdrdata/ComfyUI-Manager"
    "https://github.com/cubiq/ComfyUI_essentials"
    "https://github.com/rgthree/rgthree-comfy"
)

# Model arrays
CHECKPOINT_MODELS=(
    "https://civitai.com/api/download/models/1759168?type=Model&format=SafeTensor&size=full&fp=fp16"
    "https://civitai.com/api/download/models/1943922?type=Model&format=SafeTensor&size=pruned&fp=fp16"
)

LORA_MODELS=(
)

CONTROLNET_MODELS=(
    "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors?download=true"
)


# Create directories if they don't exist
mkdir -p "${COMFYUI_DIR}/models/checkpoints"
mkdir -p "${COMFYUI_DIR}/models/loras"
mkdir -p "${COMFYUI_DIR}/models/controlnet"

# Print token values for debugging
echo "HF_TOKEN is set to: ${HF_TOKEN}"
echo "CIVITAI_TOKEN is set to: ${CIVITAI_TOKEN}"


# Download function using aria2c
function download_model() {
    local url="$1"
    local destination="$2"
    local model_type="$3"
    local filename=""

    echo "Downloading $model_type: $url"
    echo "Destination: $destination"

    if [[ -n $HF_TOKEN && $url =~ ^https://([a-zA-Z0-9_-]+\.)?huggingface\.co(/|$|\?) ]]; then
        # HuggingFace download with authorization header
        filename=$(basename "$url" | sed 's/\?.*//')
        aria2c --log-level=error \
               --header="Authorization: Bearer $HF_TOKEN" \
               --continue=true \
               --max-connection-per-server=8 \
               --split=8 \
               --min-split-size=1M \
               --summary-interval=10 \
               --dir="$destination" -o "$filename" "$url"
    elif [[ -n $CIVITAI_TOKEN && $url =~ ^https://([a-zA-Z0-9_-]+\.)?civitai\.com(/|$|\?) ]]; then
        # Civitai download with token parameter and use content-disposition for filename
        if [[ $url == *"?"* ]]; then
            download_url="${url}&token=${CIVITAI_TOKEN}"
        else
            download_url="${url}?token=${CIVITAI_TOKEN}"
        fi
        aria2c --log-level=error \
               --content-disposition \
               --continue=true \
               --max-connection-per-server=8 \
               --split=8 \
               --min-split-size=1M \
               --summary-interval=10 \
               --dir="$destination" "$download_url"
    else
        # Generic download
        filename=$(basename "$url" | sed 's/\?.*//')
        aria2c --log-level=error \
               --continue=true \
               --max-connection-per-server=8 \
               --split=8 \
               --min-split-size=1M \
               --summary-interval=10 \
               --dir="$destination" -o "$filename" "$url"
    fi

    if [ $? -eq 0 ]; then
        echo "✓ Successfully downloaded $model_type"
    else
        echo "✗ Failed to download $model_type: $url"
    fi
    echo "----------------------------------------"
}

function provisioning_get_nodes() {
    for repo in "${NODES[@]}"; do
        dir="${repo##*/}"
        path="${COMFYUI_DIR}/custom_nodes/${dir}"
        requirements="${path}/requirements.txt"
        if [[ -d $path ]]; then
            if [[ ${AUTO_UPDATE,,} != "false" ]]; then
                printf "Updating node: %s...\n" "${repo}"
                ( cd "$path" && git pull )
                if [[ -e $requirements ]]; then
                   pip install --no-cache-dir -r "$requirements"
                fi
            fi
        else
            printf "Downloading node: %s...\n" "${repo}"
            git clone "${repo}" "${path}" --recursive
            if [[ -e $requirements ]]; then
                pip install --no-cache-dir -r "${requirements}"
            fi
        fi
    done
}

# Download and install nodes
echo "Starting node downloads..."
echo "=========================="
provisioning_get_nodes

# Download checkpoint models
echo "Starting checkpoint model downloads..."
echo "========================================"
for url in "${CHECKPOINT_MODELS[@]}"; do
    download_model "$url" "${COMFYUI_DIR}/models/checkpoints" "Checkpoint Model"
done

# Download LoRA models
echo ""
echo "Starting LoRA model downloads..."
echo "================================="
for url in "${LORA_MODELS[@]}"; do
    download_model "$url" "${COMFYUI_DIR}/models/loras" "LoRA Model"
done

# Download ControlNet models
echo ""
echo "Starting ControlNet model downloads..."
echo "================================="
for url in "${CONTROLNET_MODELS[@]}"; do
    download_model "$url" "${COMFYUI_DIR}/models/controlnet" "ControlNet Model"
done

echo ""
echo "All downloads completed!"
echo "========================"
echo "Checkpoint models saved to: ${COMFYUI_DIR}/models/checkpoints"
echo "LoRA models saved to: ${COMFYUI_DIR}/models/loras"
echo "ControlNet models saved to: ${COMFYUI_DIR}/models/controlnet"


# Optional: List downloaded files
echo ""
echo "Downloaded checkpoint models:"
ls -la "${COMFYUI_DIR}/models/checkpoints/"
echo ""
echo "Downloaded LoRA models:"
ls -la "${COMFYUI_DIR}/models/loras/"
echo ""
echo "Downloaded ControlNet models:"
ls -la "${COMFYUI_DIR}/models/controlnet/"

In [ ]:
!python main.py --dont-print-server